In [70]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sympy as sp
import old as pi

In [71]:
import sympy as sp

# Definir variables simbólicas
alpha, chi, mu, k, x, phi_is, phi_r = sp.symbols('alpha chi mu k x phi_is phi_r', real=True)
A_isx, A_isy, A_rx, A_ry = sp.symbols('A_isx A_isy A_rx A_ry', real=True)

# Definir vectores de amplitud
A_is = sp.Matrix([[A_isx], [A_isy * sp.exp(1j * phi_is)]])
A_r = sp.Matrix([[A_rx], [A_ry * sp.exp(1j * phi_r)]])

# Definir los coeficientes de la matriz de la lámina de onda
S_xx = sp.cos(alpha)**2 + sp.exp(-1j * chi) * sp.sin(alpha)**2
S_xy = (1 - sp.exp(-1j * chi)) * sp.cos(alpha) * sp.sin(alpha)
S_yx = S_xy
S_yy = sp.sin(alpha)**2 + sp.exp(-1j * chi) * sp.cos(alpha)**2

# Transformar el vector A_is con la lámina de onda
E_sx = S_xx * A_isx + S_xy * A_isy * sp.exp(1j * phi_is)
E_sy = S_yx * A_isx + S_yy * A_isy * sp.exp(1j * phi_is)
E_s = sp.Matrix([[E_sx], [E_sy]])

# Expresiones para la intensidad
Omega = A_r.norm()**2 + E_s.norm()**2
Psi = sp.Abs(A_r.dot(E_s))

# Cálculo de la fase relativa (vartheta)
numerator = -sp.Abs(A_r[1,0]) * sp.Abs(E_s[1]) * sp.sin(sp.arg(A_r[1,0]) - sp.arg(E_s[1]))
denominator = sp.Abs(A_r[0,0]) * sp.Abs(E_s[0]) + sp.Abs(A_r[1,0]) * sp.Abs(E_s[1]) * sp.cos(sp.arg(A_r[1,0]) - sp.arg(E_s[1]))
vartheta = sp.atan2(numerator, denominator)

# Modulación de fase
zeta = 2 * (k * x - vartheta)
cos_term = sp.cos(mu + zeta)


#Expresión simbólica para la intensidad
I = Omega + Psi * cos_term

In [72]:
I

A_isx**2*sin(alpha)**4 + 2*A_isx**2*sin(alpha)**2*cos(alpha)**2 + A_isx**2*cos(alpha)**4 + A_isy**2*sin(alpha)**4 + 2*A_isy**2*sin(alpha)**2*cos(alpha)**2 + A_isy**2*cos(alpha)**4 + A_rx**2 + A_ry**2 + sqrt(A_isx**2*A_rx**2*exp(1.0*I*chi)*sin(alpha)**2*cos(alpha)**2 + A_isx**2*A_rx**2*sin(alpha)**4 + A_isx**2*A_rx**2*cos(alpha)**4 + A_isx**2*A_rx**2*exp(-1.0*I*chi)*sin(alpha)**2*cos(alpha)**2 + A_isx**2*A_rx*A_ry*exp(1.0*I*chi)*exp(1.0*I*phi_r)*sin(alpha)**3*cos(alpha) - A_isx**2*A_rx*A_ry*exp(1.0*I*chi)*exp(-1.0*I*phi_r)*sin(alpha)*cos(alpha)**3 - A_isx**2*A_rx*A_ry*exp(1.0*I*phi_r)*sin(alpha)**3*cos(alpha) + A_isx**2*A_rx*A_ry*exp(1.0*I*phi_r)*sin(alpha)*cos(alpha)**3 - A_isx**2*A_rx*A_ry*exp(-1.0*I*phi_r)*sin(alpha)**3*cos(alpha) + A_isx**2*A_rx*A_ry*exp(-1.0*I*phi_r)*sin(alpha)*cos(alpha)**3 - A_isx**2*A_rx*A_ry*exp(-1.0*I*chi)*exp(1.0*I*phi_r)*sin(alpha)*cos(alpha)**3 + A_isx**2*A_rx*A_ry*exp(-1.0*I*chi)*exp(-1.0*I*phi_r)*sin(alpha)**3*cos(alpha) - A_isx**2*A_ry**2*exp(1.0*I*chi)*

In [73]:
lambda_ = 532e-9

# Ejemplo de evaluación
values = {
    alpha: 0,
    chi: 0, 
    mu: 0,
    k: 2 * sp.pi /lambda_, 
    x: 'x',
    phi_is: sp.pi/2,
    phi_r: sp.pi/2,
    A_isx: 1/sp.sqrt(2),
    A_isy: 1/sp.sqrt(2),
    A_rx: 1/sp.sqrt(2),
    A_ry: 1/sp.sqrt(2)
}

I_evaluated = I.subs(values).evalf()
I_evaluated



2.00000000000000

In [78]:
# Parámetros físicos
lambda_ = 532e-9  # Longitud de onda en metros
x_size, y_size = 10, 5  # Dimensiones

# Escalas de conversión de píxeles a metros
x_scale, y_scale = lambda_ / x_size, lambda_ / y_size

# Crear coordenadas físicas directamente
X_meters, Y_meters = np.meshgrid(np.arange(x_size) * x_scale, np.arange(y_size) * y_scale)

# Dimensión combinada en coordenadas físicas
dim = X_meters + Y_meters

values = {
    alpha: 0,
    chi: 0,
    mu: 0,
    k: 2 * sp.pi / lambda_,
    x: x,  # Se evalúa luego directamente sobre la malla
    phi_is: sp.pi/2,
    phi_r: sp.pi/2,
    A_isx: 1/sp.sqrt(2),
    A_isy: 1/sp.sqrt(2),
    A_rx: 1/sp.sqrt(2),
    A_ry: 1/sp.sqrt(2)
}

# Optimización: Evaluar la expresión completa sobre la malla
I_evaluated = np.vectorize(lambda px: I.subs(values).subs(x, px).evalf())(dim)

